# 데이터 받아오기

In [1]:
from modules.scouter_handler import ScouterHandler
from modules.doc_graph_analyzer import DocGraphAnalyzer
import numpy as np

In [2]:
keyword = "대통령"

min_rel_score = 2.0
max_length = 3000

In [3]:
scouter = ScouterHandler(size=500)

In [4]:
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'node', 'edge', 'extContent'])
doc_info_list = scouter.search(query_body, data_type="graphs", trim_lower=True)

Query : 대통령
Scroll idx : 1 (500 docs)
Scroll idx : 2 (500 docs)
Scroll idx : 3 (500 docs)
Scroll idx : 4 (500 docs)
Scroll idx : 5 (500 docs)
Scroll idx : 6 (500 docs)
Scroll idx : 7 (500 docs)
Scroll idx : 8 (475 docs)
Trim lower 70% of docs (1192 docs are cut)
Total retrieved Doc # :  2782



In [5]:
doc_info_list = [d for d in doc_info_list if d['rel_score'] > min_rel_score and len(d['extContent']) < max_length]
num_doc = len(doc_info_list)
num_doc

2761

In [6]:
doc_temp_id_list = {doc_info['news_id']:i for i, doc_info in enumerate(doc_info_list)}

# 문서 클러스터링

In [7]:
# It is implemented to add hierarchical clustering, document-node clustering, soft-clustering 
doc_graph_anal = DocGraphAnalyzer(keyword, df_bound=(0.01, 0.80))
doc_graph_anal.extract_subtopic_info(doc_info_list, top_doc_num=5, top_keyword_num=10,
                                     hierarchy=True, cutting_level=3)

Doc #  : 2761
Node # : 1669
Edge # : 330990

Community 0 :  399 Keywords, (검찰/수사/조사/관련/혐의/의혹/의원/이명박/자금/소환)
	Community 0/0 :   76 Keywords, (검찰/조사/소환/수사/이명박/통보/진행/피의/출석/필요)
		Leaf Community 0/0/2 :   35 Keywords, (검찰/조사/소환/이명박/수사/통보/관측/처리/착수/방침) top5 documents, ([67713, 61306, 19152, 22675, 22745])
	Community 0/1 :  104 Keywords, (혐의/구속/특수/사건/서울중앙/뇌물/부장/재판/오전/활동)
		Leaf Community 0/1/1 :   42 Keywords, (구속/재판/사건/기소/징역/법원/선고/영장/농단/비리) top5 documents, ([93253, 75475, 96054, 24075, 71909])
		Leaf Community 0/1/3 :   28 Keywords, (지검/서울중앙/부장/특수/검사/오전/송경호/증거/범죄/첨단) top5 documents, ([42282, 42054, 44394, 71909, 60174])
	Community 0/2 :   77 Keywords, (의원/자금/의혹/회장/불법/과정/대선/금품/이상득/수수)
		Leaf Community 0/2/0 :   20 Keywords, (자금/의혹/대선/불법/과정/수수/역할/관여/민간/핵심) top5 documents, ([26660, 36965, 37323, 38032, 37398])
		Leaf Community 0/2/1 :   23 Keywords, (회장/금융/이팔성/지주/금품/이상득/인사/명목/측근/전달) top5 documents, ([44345, 44519, 55392, 90146, 90150])
		Leaf Community 0/2/2 :   25 Keywords, (공천/헌금/의심/김소남/정황/제공/포

# json 파일 저장
* 결과 Dictionary
	- key(depth val) : Hierarchical clustering에서 계층 구조의 depth (예, 0 또는 0/1)
	- value : dictionary
			- key('Keywords') = 해당 depth 클러스터의 top 키워드 10
			- key('documents') = 해당 depth 클러스터의 관련도로 랭킹된 문서 인덱스
			- key('docs_rel') = 해당 depth 클러스터의 관련도로 랭킹된 문서의 관련도 값
			- key('volume') = 해당 depth 클러스터의 키워드 개수
			

In [9]:
doc_graph_anal.save_result_tojson(json_path='output.json')

# 결과 분석

In [11]:
chosen_docidx = 54458   
print(doc_info_list[doc_temp_id_list[chosen_docidx]]['extContent'])

성달표  현대통상 회장이 5일 열린 제52회 납세자의 날 기념식에서  아름다운 납세자상 을 수상했다. 성 회장은 지난 2003년 현대모비스 부품지원센터인  현대통상과 현대모비스 협력업체인  에스더블유,  에스에이엠,  에스디피 등을 경영해 오면서 성실한 납세와 지역사회를 위해 기부와 봉사활동을 헌신적으로 해온 사실이 알려지면서 아름다운 납세자상을 수상하게 됐다. 그는 현대모비스 전국부품지원센터협의회 회장과 경산상공회의소 상임의원, 경산시체육회 상임부회장, 대구시양궁협회 부회장, 대구가톨릭경제인회 부회장, 경북대총동창회 부회장 등을 맡아 활발한 사회활동을 하고 있다. 성 회장은 특히 지역경제발전과 일자리 창출에 크게 기여하면서도 경산과 포항, 대구 등지의 여러 장학회와 대학생들에게 인재양성 장학금을, 대학에는 대학발전기금을 전달해 오고 있다. 불우이웃돕기 성금, 체육회 진흥기금, 가톨릭재단 후원금, 장애인후원회 후원금, 매일신문 주최 효 잔치와 호국의 달 감사음악회 후원금 등을 전달하고, 출소자 합동결혼식 후원 등 다양한 형태의 기부와 봉사활동도 꾸준히 실천하고 있다. 성 회장은 이 같은 공로를 인정받아 지난해 12월 민주평통 의장(대통령) 표창, 2009년 제43회 납세자의 날 성실납세자로 뽑혀 대구지방국세청장 표창, 제39회 저축의 날 금융감독원장 표창, 통일부 장관상, 대구시 모범시민상 등 다양한 표창과 상을 받았다. 성 회장은  국가와 사회를 위해 더 훌륭한 일과 기부, 봉사를 하는 분이 많은데 아름다운 납세자상을 수상하게 돼 더없이 영광스럽고 책임감도 느낀다 면서  앞으로도 기업경영을 잘해서 성장시키고, 이익을 사회에 환원하고, 이웃을 위해 봉사하는 삶을 살도록 노력하겠다 고 약속했다.


In [12]:
a = doc_graph_anal.relevance_obj.np_docsNsubtopic_rel
a[doc_temp_id_list[chosen_docidx]] / sum(a[doc_temp_id_list[chosen_docidx]])

array([0.        , 0.        , 0.        , 0.        , 0.01917099,
       0.        , 0.01234365, 0.02897733, 0.        , 0.01717492,
       0.        , 0.        , 0.        , 0.        , 0.00212684,
       0.        , 0.        , 0.        , 0.15750715, 0.        ,
       0.        , 0.02976173, 0.02046078, 0.        , 0.00407927,
       0.02914638, 0.00615683, 0.        , 0.        , 0.05687838,
       0.02290009, 0.        , 0.00326159, 0.        , 0.06016417,
       0.16088341, 0.01432683, 0.2156207 , 0.13905898])

# Scouter를 사용한 결과 분석

In [ ]:
top_doc_id_query_list = [scouter.make_doc_id_query_body(top_doc_ids, filters=["news_id", "newsTitle", "extContent"])
                         for top_doc_ids in doc_graph_anal.top_doc_id_list]
top_doc_info_list = [scouter.search(doc_id_query, data_type='newspaper', trim_lower=False, silence=True)
                     for doc_id_query in top_doc_id_query_list]